<a href="https://colab.research.google.com/github/sokrypton/py2Dmol/blob/main/py2Dmol_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install py2Dmol

In [1]:
import py2Dmol

In [2]:
py2Dmol.view(autoplay=True).from_pdb('1YNE')

In [3]:
py2Dmol.view(rotate=True).from_pdb('1BJP', use_biounit=True, ignore_ligands=True)

In [4]:
py2Dmol.view((500,500), controls=False, box=False).from_pdb('9D2J')

In [5]:
py2Dmol.view(pae=True).from_afdb('Q5VSL9')

In [8]:
import numpy as np
def helix(n, radius=2.3, rise=1.5, rotation=100):
    """Generate helical coordinates."""
    angles = np.radians(rotation) * np.arange(n)
    return np.column_stack([
        radius * np.cos(angles),
        radius * np.sin(angles),
        rise * np.arange(n)
    ])

# Protein helix (50 residues)
protein = helix(50)
protein[:, 0] += 15  # offset x

# DNA strand (30 bases)
dna = helix(30, radius=10, rise=3.4, rotation=36)
dna[:, 0] -= 15  # offset x

# Ligand ring (6 atoms)
angles = np.linspace(0, 2*np.pi, 6, endpoint=False)
ligand = np.column_stack([
    1.4 * np.cos(angles),
    1.4 * np.sin(angles),
    np.full(6, 40)
])

# Combine everything (86 atoms total)
coords = np.vstack([protein, dna, ligand])
plddts = np.concatenate([np.full(50, 90), np.full(30, 85), np.full(6, 70)])
chains = ['A']*50 + ['B']*30 + ['L']*6
types = ['P']*50 + ['D']*30 + ['L']*6

viewer = py2Dmol.view((400,300),rotate=True)
viewer.add(coords, plddts, chains, types)
viewer.show()

In [9]:
import numpy as np
import time

# Define the wiggle function
def circle_morph(n=20, wave=0):
    """n points, constant ~3.8Å bonds, wavy deformation."""
    bond = 3.8
    perimeter = n * bond
    radius = perimeter / (2 * np.pi)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False)
    r = radius * (1 + wave * 0.2 * np.sin(4 * angles))
    return np.column_stack([
        r * np.cos(angles),
        r * np.sin(angles),
        wave * 3 * np.cos(angles)
    ])

# 1. Create the viewer object
viewer = py2Dmol.view()

# 2. Show the viewer *before* adding data to enter "Live Mode"
viewer.show()

# 3. Now, add frames in a loop
for frame in range(60):
    w = np.sin(frame * np.pi / 15)
    coords = circle_morph(20, w)
    plddts = np.full((20,), 80.0)
    chains = ['A'] * 20
    atom_types = ['P'] * 20

    # Send the new frame to the live viewer
    viewer.add(coords, plddts, chains, atom_types)

    # Wait a bit
    time.sleep(0.1)

In [10]:
import numpy as np
import time
viewer = py2Dmol.view((400,300),pae=True)
for _ in range(10):
  coords = np.random.normal(size=(50,3))
  coords[:,1] += np.arange(50)
  viewer.add(coords, pae=np.random.uniform(0,30,(50,50)))
  time.sleep(0.1)
viewer.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

viewers = []
n = 0
for rotate in [True,False]:
  for autoplay in [True,False]:
      output = widgets.Output()
      with output:
          reuse_js = n > 0
          py2Dmol.view(controls=False, box=False, rotate=rotate,
                       autoplay=autoplay, reuse_js=reuse_js).from_pdb('1YNE')
          n += 1
      viewers.append(output)

# Create the grid, 2 columns, auto-sizing rows
grid = widgets.GridBox(viewers, layout=widgets.Layout(
    grid_template_columns="repeat(2, 300px)"
))

display(grid)

In [12]:
with py2Dmol.grid(2,2,size=(200,200)) as g:
    for rotate in [True,False]:
      for autoplay in [True,False]:
        g.view(rotate=rotate, autoplay=autoplay).from_pdb("1YNE")